In [1]:
import os
import re
import numpy as np
import pandas as pd
from IPython.display import Image 
from numpy import load, savez
import torch
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from statistics import mean
from sklearn.preprocessing import normalize
import sys
import os
#sys.path.append(os.path.dirname(os.path.abspath(__file__)))


In [2]:
import subprocess
from numpy import load, savez
from IPython.display import Image
from IPython import display
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize

def optional_print(text, verbosity=True):
    if verbosity:
        print(text)
    else:
        ...

def getByUniqueIdEmotion(df, idUnique, emotion):
    # Todos los registros donde ID está EMOCION
    gender = idUnique[-1]
    id = idUnique[:-1]
    res = df.loc[(df['exp'] == emotion) & (df['idUnique'] == idUnique)]
    
    # Todos los registros donde ID está EMOCION, ordenados de menos EMOCION a más EMOCION
    res = res.sort_values(by=['exp_level'],ascending=True, inplace=False)
    res.reset_index(drop=True, inplace=True)
    return res

def getNPZ(df, index):
    # filename = df['projected_file'][index]
    # TODO: volver a load(filename)['w']
    fileName = df['name'][index]
    filenameFinal = f'/mnt/discoAmpliado/viky/images/results_BU_3DFE/{fileName}/projected_w.npz'
    return load(filenameFinal)['w']

In [ ]:
def create_image_dir(base_dir, verbosity=True):
    if not os.path.exists(base_dir):
        optional_print("La ruta no existe:" + base_dir, verbosity)
    else:
        optional_print("Ruta encontrada:" + base_dir, verbosity)

    # Regex pattern to match the filenames
    pattern = re.compile(r'^(?P<gender>[MF])(?P<id>\d{4})_(?P<exp>NE|AN|DI|FE|HA|SA|SU)(?P<exp_level>00|01|02|03|04)(?P<race>WH|BL|IN|AE|AM|LA)_(?P<attribute>F2D)\.(?P<ext>bmp)$')
    # Dictionary to hold the image data
    images_data = []

    # Walk through the base directory
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            match = pattern.match(file)
            if match and match.group('ext') == 'bmp':
                image_data = match.groupdict()
                image_data['name'] = os.path.splitext(file)[0]
                image_data['raw_image_folder'] = root
                image_data['file'] = file
                image_data['projected_npz'] = "N"
                image_data['projected_file'] = "N"
                image_data['idUnique'] = image_data['id'] + image_data['gender']
                images_data.append(image_data)
    return images_data

In [ ]:
from sklearn.linear_model import LinearRegression

def linearRegression(df, idUnique, emotion, verbose=True):

    # 1. Preparar los datos
    data = getByUniqueIdEmotion(df, idUnique, emotion)
    
    # Supongamos que tienes los siguientes datos:
    vectores_latentes = [getNPZ(data, index) for index in data.index] # Lista de vectores latentes, cada uno con forma (1, 18, 512)
    niveles_emocion = [1, 2, 3, 4]  # Lista de niveles de emocion asociados, cada uno es un escalar
    
    # Convertir los vectores latentes a una matriz de forma (n_samples, 18 * 512)
    X = np.array([vector.reshape(-1) for vector in vectores_latentes])  # Aplana cada vector latente a 1D
    y = np.array(niveles_emocion)  # Convertir los niveles de felicidad a un array numpy
    
    # 2. Ajustar el modelo de regresión lineal
    
    # Crear el modelo de regresión lineal
    modelo = LinearRegression()
    
    # Ajustar el modelo a los datos
    modelo.fit(X, y)
    
    # Obtener la dirección de emocion en el espacio latente
    direccion_emocion = modelo.coef_.reshape(1, 18, 512)

    optional_print(f'Executed Linear Regression for person: {idUnique}, emotion: {emotion}.', verbose)
    return direccion_emocion

In [ ]:
gc.collect()
torch.cuda.empty_cache()
verbosity = True

# Define the base directory
base_dir = "/mnt/discoAmpliado/viky/BU_3DFE"

images_data = create_image_dir(base_dir, verbosity)

# TODO: Hacer andar el docker que proyecta
##align_images(images_data[0:2], verbosity)
##df = batch_processing(images_data[0:2], verbosity)
# df = load_dataframe()

# TODO: A partir de acá sigo como si ya tuviera las imágenes proyectadas, porque sino no llego a ningún lado.
df = pd.read_csv(f'/mnt/discoAmpliado/viky/dataframes/processed_dataframe_combined_fallback.csv')
df['idUnique'] = df['id'].astype(str) + df['gender']

# Crear lista de emociones y eliminar 'NE'
emociones = df.exp.unique().tolist()
emociones.remove("NE")

# Crear lista de IDs y eliminar IDs no deseados
ids = df.idUnique.unique().tolist()
ids_malos = ["39M", "17M", "22M", "14M", "2F"] # TODO: Sacar esto cuando pueda arreglar el docker
for id_malo in ids_malos:
    ids.remove(id_malo)

# Crear un DataFrame vacío con IDs como índice y emociones como columnas
emociones_total_LR = pd.DataFrame(index=ids, columns=emociones)
emociones_total_PCA = pd.DataFrame(index=ids, columns=emociones)

# Calcular el vector de cada emoción para cada persona
for emocion in emociones:
    for idUnique in ids:
        # LR
        direccion_emocion = linearRegression(df, idUnique, emocion, False)
        emociones_total_LR.at[idUnique, emocion] = direccion_emocion.flatten()
        
        # PCA
        ##direccion_emocion = executePCA(df, idUnique, emocion, False)
        ##emociones_total_PCA.at[idUnique, emocion] = direccion_emocion

In [ ]:
df.iloc()[0]
getNPZ(df, 0).shape

In [ ]:
import numpy as np

def restar_neutro(df, vectores):
    """
    Resta el vector de expresión neutra a cada imagen de la misma persona.

    Parámetros:
    - df: DataFrame con información de las imágenes.
    - vectores: Diccionario con los vectores de cada imagen. {nombre_imagen: vector}
    
    Retorna:
    - nuevo_diccionario: {nombre_imagen: vector_transformado}
    """
    nuevo_diccionario = {}
    
    for idUnique in df['idUnique'].unique():
        # Obtener el vector neutro de la persona
        fila_neutra = df[(df['idUnique'] == idUnique) & (df['exp'] == 'NE')]  # NE sería neutral
        if fila_neutra.empty:
            continue
        vector_neutro = vectores[fila_neutra.iloc[0]['name']]

        # Restarlo a las otras expresiones
        for _, fila in df[df['idUnique'] == idUnique].iterrows():
            nombre_imagen = fila['name']
            if nombre_imagen in vectores:
                nuevo_diccionario[nombre_imagen] = vectores[nombre_imagen] - vector_neutro

    return nuevo_diccionario


In [ ]:
from sklearn.preprocessing import normalize

def obtener_vectores_promediados(df, vectores_neutralizados):
    """
    Promedia los 4 niveles de cada emoción para cada persona.

    Parámetros:
    - df: DataFrame con información de las imágenes.
    - vectores_neutralizados: Diccionario con los vectores restados.

    Retorna:
    - diccionario_promediado: {idUnique: {emocion: vector_promediado}}
    """
    diccionario_promediado = {}

    for idUnique in df['idUnique'].unique():
        diccionario_promediado[idUnique] = {}

        for emocion in df['exp'].unique():
            if emocion == 'NE':  # No considerar la expresión neutra
                continue

            # Obtener los vectores de los 4 niveles
            filas = df[(df['idUnique'] == idUnique) & (df['exp'] == emocion)]
            vectores = [vectores_neutralizados[fila['name']] for _, fila in filas.iterrows()]

            # Normalizar cada vector y luego promediar
            if vectores:
                vectores = np.array(vectores)
                vectores_normalizados = normalize(vectores, axis=1)  # Normaliza cada vector individualmente
                vector_promedio = np.mean(vectores_normalizados, axis=0)

                diccionario_promediado[idUnique][emocion] = vector_promedio

    return diccionario_promediado


In [ ]:
from sklearn.linear_model import LinearRegression

def calcular_direccion_emocion(df, vectores_promediados):
    """
    Usa regresión lineal para encontrar la dirección de cada emoción.

    Parámetros:
    - df: DataFrame con información de las imágenes.
    - vectores_promediados: {idUnique: {emocion: vector}}

    Retorna:
    - direcciones: {emocion: vector_direccion}
    """
    direcciones = {}

    for emocion in df['exp'].unique():
        if emocion == 'NE':
            continue

        X = []
        for idUnique in vectores_promediados:
            if emocion in vectores_promediados[idUnique]:
                X.append(vectores_promediados[idUnique][emocion].flatten())  # Asegurarse de que sean arrays 1D

        if X:
            X = np.array(X)
            y = np.ones(len(X))  # No importa realmente el valor de y, porque nos interesa la dirección

            modelo = LinearRegression(fit_intercept=False)
            modelo.fit(X, y)

            direcciones[emocion] = modelo.coef_

    return direcciones


In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression

def load_latent_vector(name):
    """Carga el vector latente desde un archivo .npz basado en el nombre de la imagen."""
    path = f'/mnt/discoAmpliado/viky/images/results_BU_3DFE/{name}/projected_w.npz'
    if not os.path.exists(path):
        raise FileNotFoundError(f"El archivo {path} no existe.")
    data = np.load(path)
    return data['w']  # Suponiendo que la clave dentro del .npz es 'w'

def normalize_vectors(vectors):
    """Normaliza una lista de vectores usando norma L2."""
    original_shape = vectors.shape  # Guardamos la forma original
    vectors = vectors.reshape(vectors.shape[0], -1)  # Aplanamos a (N, D)
    vectors = normalize(vectors, axis=1)  # Normalizamos en la dimensión correcta
    return vectors.reshape(original_shape)  # Restauramos la forma original

def method_1_average_then_regression(df):
    """Método 1: Promedia los vectores por emoción y aplica regresión."""
    results = {}
    emotions = ['DI', 'HA', 'SU', 'AN', 'SA', 'FE']  # Suponiendo estas emociones
    
    for emotion in emotions:
        subset = df[df['exp'] == emotion]
        vectors = np.array([load_latent_vector(name) for name in subset['name']])
        vectors = normalize_vectors(vectors)
        avg_vector = np.mean(vectors, axis=0)
        
        X = np.arange(len(vectors)).reshape(-1, 1)
        y = vectors.reshape(vectors.shape[0], -1)  # Aplanamos los vectores antes de la regresión

        model = LinearRegression().fit(X, y)
        results[emotion] = model.coef_.reshape(1, 18, 512)  # Restauramos la forma original
    
    return results

def method_2_regression_by_emotion_and_level(df):
    """Método 2: Aplica regresión a cada emoción y nivel de intensidad."""
    results = {}
    emotions = ['DI', 'HA', 'SU', 'AN', 'SA', 'FE']
    
    for emotion in emotions:
        results[emotion] = {}  # Ahora almacenamos los resultados por nivel
        
        for level in sorted(df['exp_level'].unique()):
            subset = df[(df['exp'] == emotion) & (df['exp_level'] == level)]
            if subset.empty:
                continue
            vectors = np.array([load_latent_vector(name) for name in subset['name']])
            vectors = normalize_vectors(vectors)
            
            X = np.arange(len(vectors)).reshape(-1, 1)
            y = vectors.reshape(vectors.shape[0], -1)  # Aplanamos los vectores antes de la regresión
            
            model = LinearRegression().fit(X, y)
            results[emotion][level] = model.coef_.reshape(1, 18, 512)  # Guardamos cada nivel separadamente
    
    return results

def method_3_regression_with_level_variable(df):
    """Método 3: Incluye el nivel como variable numérica en la regresión."""
    results = {}
    emotions = ['DI', 'HA', 'SU', 'AN', 'SA', 'FE']
    
    for emotion in emotions:
        subset = df[df['exp'] == emotion]
        vectors = np.array([load_latent_vector(name) for name in subset['name']])
        vectors = normalize_vectors(vectors)
        levels = subset['exp_level'].values.reshape(-1, 1)

        y = vectors.reshape(vectors.shape[0], -1)
        
        model = LinearRegression().fit(levels, y)
        results[emotion] = model.coef_.reshape(1, 18, 512)
    
    return results

def save_results_as_csv(results, filename):
    flat_results = {}
    
    for key, value in results.items():
        flat_results[key] = value.flatten()  # Aplanar la matriz
    
    df = pd.DataFrame.from_dict(flat_results, orient='index')
    df.to_csv(filename)

    save_results_as_csv(method_1_results, 'method_1_results.csv')


def save_results_as_npz(results, filename):
    """Guarda los resultados en un archivo NPZ."""
    np.savez(filename, **results)

# Cargar datos y ejecutar procesos
df = pd.read_csv('/mnt/discoAmpliado/viky/dataframes/processed_dataframe_combined_fallback.csv')  # Ajustar la ruta al dataset
df['idUnique'] = df['id'].astype(str) + df['gender']
ids_malos = ["39M", "17M", "22M", "14M", "2F"]
df = df[~df['idUnique'].isin(ids_malos)]

method_1_results = method_1_average_then_regression(df)
method_2_results = method_2_regression_by_emotion_and_level(df)
method_3_results = method_3_regression_with_level_variable(df)

print("Resultados método 1, 'DI': " + str(method_1_results['DI'].shape))
print("Resultados método 2, 'DI': " + str(method_2_results['DI'][1].shape))
print("Resultados método 3, 'DI': " + str(method_3_results['DI'].shape))

# Guardar resultados
##save_results_as_csv(method_1_results, 'method_1_results.csv')
save_results_as_npz(method_1_results, 'method_1_results.npz')
##save_results_as_csv(method_2_results, 'method_2_results.csv')
save_results_as_npz(method_2_results, 'method_2_results.npz')
##save_results_as_csv(method_3_results, 'method_3_results.csv')
save_results_as_npz(method_3_results, 'method_3_results.npz')


Resultados método 1, 'DI': (1, 18, 512)
Resultados método 2, 'DI': (1, 18, 512)
Resultados método 3, 'DI': (1, 18, 512)


In [3]:
def print_shape(diccionario, indent=0):
    for clave, valor in diccionario.items():
        # Indentación para la estructura
        print("  " * indent + f"Clave: {clave}", end=' ')
        
        if isinstance(valor, dict):
            # Si el valor es otro diccionario, lo recorremos recursivamente
            print("; Valor: Diccionario con las siguientes claves: ")
            print_shape(valor, indent + 1)
        elif isinstance(valor, np.ndarray):
            # Si el valor es un array de NumPy, mostramos su forma
            print(f"; Valor: Array de la forma {valor.shape}")
        elif isinstance(valor, list):
            # Si el valor es una lista, mostramos su longitud
            print(f"(Lista) - Shape: {len(valor)}")
        else:
            # Si no es ni diccionario, ni array ni lista, solo mostramos el tipo
            print(f"({type(valor).__name__})")

print("Forma del diccionario correspondiente al método 1: ")
print_shape(method_1_results)
print("Fin del diccionario correspondiente al método 1.\n")
print("Forma del diccionario correspondiente al método 2: ")
print_shape(method_2_results)
print("Fin del diccionario correspondiente al método 2.\n")
print("Forma del diccionario correspondiente al método 3: ")
print_shape(method_3_results)
print("Fin del diccionario correspondiente al método 3.\n")

Forma del diccionario correspondiente al método 1: 
Clave: DI ; Valor: Array de la forma (1, 18, 512)
Clave: HA ; Valor: Array de la forma (1, 18, 512)
Clave: SU ; Valor: Array de la forma (1, 18, 512)
Clave: AN ; Valor: Array de la forma (1, 18, 512)
Clave: SA ; Valor: Array de la forma (1, 18, 512)
Clave: FE ; Valor: Array de la forma (1, 18, 512)
Fin del diccionario correspondiente al método 1.

Forma del diccionario correspondiente al método 2: 
Clave: DI ; Valor: Diccionario con las siguientes claves: 
  Clave: 1 ; Valor: Array de la forma (1, 18, 512)
  Clave: 2 ; Valor: Array de la forma (1, 18, 512)
  Clave: 3 ; Valor: Array de la forma (1, 18, 512)
  Clave: 4 ; Valor: Array de la forma (1, 18, 512)
Clave: HA ; Valor: Diccionario con las siguientes claves: 
  Clave: 1 ; Valor: Array de la forma (1, 18, 512)
  Clave: 2 ; Valor: Array de la forma (1, 18, 512)
  Clave: 3 ; Valor: Array de la forma (1, 18, 512)
  Clave: 4 ; Valor: Array de la forma (1, 18, 512)
Clave: SU ; Valor: D

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.spatial.distance import cosine
import seaborn as sns

# Función para calcular la similitud coseno entre vectores
def cosine_similarity(v1, v2):
    return 1 - cosine(v1.flatten(), v2.flatten())

# Estrategia 1: Comparación de similitud coseno
# Explicación: Calculamos la similitud coseno entre los vectores de cada emoción para evaluar si los métodos producen resultados similares.
def compare_cosine_similarity(results1, results2, results2_avg):
    similarities = {}
    for emotion in results1.keys():
        sim_1_3 = cosine_similarity(results1[emotion], results2_avg[emotion])
        sim_2_3 = np.mean([cosine_similarity(results2[emotion][level], results2_avg[emotion]) for level in results2[emotion]])
        similarities[emotion] = (sim_1_3, sim_2_3)
    return similarities


In [5]:
# Estrategia 2: Comparación de dispersión
# Explicación: Medimos la varianza de los valores en cada vector para evaluar el nivel de ruido en cada método.
def compute_variance(results):
    variances = {emotion: np.var(vector) for emotion, vector in results.items()}
    return variances


In [6]:
# Estrategia 4: Visualización con PCA
# Explicación: Reducimos la dimensionalidad de los vectores latentes y graficamos para ver cómo se agrupan los métodos.
def plot_pca(results1, results2_avg, results3):
    data = []
    labels = []
    methods = []
    
    for emotion in results1.keys():
        data.append(results1[emotion].flatten())
        labels.append(emotion)
        methods.append("Método 1")
        
        data.append(results2_avg[emotion].flatten())
        labels.append(emotion)
        methods.append("Método 2")
        
        data.append(results3[emotion].flatten())
        labels.append(emotion)
        methods.append("Método 3")
    
    pca = PCA(n_components=2)
    transformed = pca.fit_transform(data)
    
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=transformed[:, 0], y=transformed[:, 1], hue=labels, style=methods, s=100)
    plt.title("Visualización PCA de vectores latentes por método")
    plt.show()

In [7]:
# Estrategia 5: Test de estabilidad con Bootstrapping
# Explicación: Muestreamos aleatoriamente subconjuntos de datos y evaluamos la varianza de los resultados para ver qué método es más estable.
def bootstrap_stability(results, num_samples=100):
    stability = {}
    for emotion in results.keys():
        boot_samples = []
        for _ in range(num_samples):
            sampled_indices = np.random.choice(results[emotion].shape[1], size=results[emotion].shape[1], replace=True)
            sampled_vector = results[emotion][:, sampled_indices, :]
            boot_samples.append(sampled_vector.flatten())
        
        boot_samples = np.array(boot_samples)
        stability[emotion] = np.var(boot_samples)
    return stability

### Similitud Coseno

Cómo interpretar los resultados

- Un valor cercano a 1 indica que los vectores son casi idénticos.
- Un valor cercano a 0 indica que son muy diferentes.
- Si el método 2 tiene valores similares a los otros, significa que promediar los niveles no cambia tanto la dirección final del vector.

In [8]:
cosine_results = compare_cosine_similarity(method_1_results, method_2_results, method_3_results)
print(cosine_results)







CHATGPT DIJO ESTO: 🔹 Cómo ejecutarlo
Llamás a la función con los resultados de cada método:

python
Copiar
Editar
cosine_results = compare_cosine_similarity(results_method_1, results_method_2, results_method_2_avg)
Esto devuelve un diccionario con la similitud coseno entre:

Método 1 y Método 2 Promediado
Cada nivel del Método 2 y su versión promediada
🔹 Cómo interpretar los resultados
Valores cercanos a 1 → Los métodos producen resultados similares.
Valores cercanos a 0 → Hay diferencias significativas.
Si el Método 2 promediado es muy diferente de sus niveles originales, significa que promediar puede estar perdiendo información importante.


SyntaxError: invalid syntax (124842550.py, line 10)

### Comparación de Dispersión (Varianza)

Cómo interpretar los resultados
- Si la varianza es alta, significa que los valores dentro del método son más dispersos → el método es más sensible a variaciones.
- Si la varianza es baja, indica que los valores están más agrupados → el método es más estable.
- Si el método 2 tiene mucha más varianza, significa que los diferentes niveles de emoción generan más ruido.

In [9]:
variance_results_1 = compute_variance(method_1_results)
variance_results_2_avg = compute_variance(method_2_results)
variance_results_3 = compute_variance(method_3_results)


AttributeError: 'dict' object has no attribute 'dtype'

### Visualización con PCA

Cómo interpretar los resultados
- Si los puntos de diferentes métodos están cercanos, significa que los métodos generan resultados similares.
- Si están muy dispersos o en grupos separados, significa que hay diferencias estructurales en los métodos.
- Si el método 2 se distribuye en diferentes regiones, significa que los niveles de emoción generan variaciones notables.

In [10]:
plot_pca(method_1_results, method_2_results, method_3_results)


AttributeError: 'dict' object has no attribute 'flatten'

### Test de Estabilidad con Bootstrapping

Cómo interpretar los resultados
- Si la estabilidad de un método es alta, significa que pequeñas variaciones en los datos no alteran mucho el resultado.
- Si es baja, el método es sensible a los cambios en los datos.
- Si el método 2 muestra menos estabilidad, significa que agregar los niveles de emoción introduce ruido.

In [11]:
bootstrap_results_1 = bootstrap_stability(method_1_results)
bootstrap_results_2_avg = bootstrap_stability(method_2_results)
bootstrap_results_3 = bootstrap_stability(method_3_results)


AttributeError: 'dict' object has no attribute 'shape'

|Estrategia	|Qué indica|	Cómo saber si un método es mejor?|
|---|---|---|
|Similitud coseno|	Qué tan parecidos son los métodos|	Valores cercanos a 1 significan similitud alta|
|Varianza|	Si hay ruido en los resultados|	Varianza baja significa mayor estabilidad|
|PCA|	Cómo se agrupan los métodos|	Métodos agrupados = resultados similares|
|Bootstrapping|	Estabilidad de los métodos|	Estabilidad alta = método más robusto|

💡 Conclusión:
Si el método 2 tiene menor estabilidad y más varianza, quizás el promedio de los niveles no ayuda tanto.
Si los tres métodos son similares en la similitud coseno y PCA, entonces cualquiera de ellos podría ser válido.

In [26]:
df.head()

,gender,id,exp,exp_level,race,attribute,ext,name,raw_image_folder,file,projected_npz,projected_file,idUnique
124,F,54,AN,1,WH,F2D,bmp,F0054_AN01WH_F2D,/home/vicky/Documents/BU_3DFE/F0054,F0054_AN01WH_F2D.bmp,[[[ 0.9724985 0.13166972 1.008977 ... 0....,images/results/BU_3DFE/F0054_AN01WH_F2D/projec...,54F
125,F,54,NE,0,WH,F2D,bmp,F0054_NE00WH_F2D,/home/vicky/Documents/BU_3DFE/F0054,F0054_NE00WH_F2D.bmp,[[[-0.78966004 2.7504096 1.835071 ... 0....,images/results/BU_3DFE/F0054_NE00WH_F2D/projec...,54F
126,F,54,SU,1,WH,F2D,bmp,F0054_SU01WH_F2D,/home/vicky/Documents/BU_3DFE/F0054,F0054_SU01WH_F2D.bmp,[[[-0.95715564 -0.6468469 0.31578222 ... -0....,images/results/BU_3DFE/F0054_SU01WH_F2D/projec...,54F
127,F,54,HA,3,WH,F2D,bmp,F0054_HA03WH_F2D,/home/vicky/Documents/BU_3DFE/F0054,F0054_HA03WH_F2D.bmp,[[[ 0.21925983 0.20221505 0.10414152 ... 0....,images/results/BU_3DFE/F0054_HA03WH_F2D/projec...,54F
128,F,54,FE,4,WH,F2D,bmp,F0054_FE04WH_F2D,/home/vicky/Documents/BU_3DFE/F0054,F0054_FE04WH_F2D.bmp,[[[ 0.08871058 -1.3138419 0.97178036 ... 1....,images/results/BU_3DFE/F0054_FE04WH_F2D/projec...,54F
